In [39]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 15 11:01:32 2022

@author: Simone
"""

import numpy as np 
import pandas as pd 
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN,KMeans
from sklearn.preprocessing import StandardScaler
import pyproj
import json
from shapely.geometry import Point, mapping
from functools import partial
from shapely.ops import transform
from mpl_toolkits.mplot3d import proj3d
from scipy.spatial.distance import cdist


%matplotlib notebook
df = pd.read_csv("New/datasetItalyComplete.csv", dtype = {"ITTER107":str})
df = df.loc[df['Sesso'] != "totale"]
df = df.loc[df['Stato civile'] != "totale"]

In [40]:
df['populPerKm2'] = df['Value']/df['Size km2']
df = df.loc[df['populPerKm2'] > 10]#Filter out small villages
df = df.reset_index().drop(["index"],axis =1)#.to_csv("datasetItalyCompleteNoSmallVill.csv", encoding='utf-8',index=False)
df

,Territorio,Sesso,Stato civile,AgeGroup,Value,lat,lng,Size km2,Redditi 2020,AvgNumberOfComponents,ITTER107,populPerKm2
0,Abano Terme,femmine,coniugata/o,30s,447,45.359387,11.788713,21.57,20753.0,2.22,28001.0,20.723227
1,Abano Terme,femmine,coniugata/o,40s,890,45.359387,11.788713,21.57,20753.0,2.22,28001.0,41.261011
2,Abano Terme,femmine,coniugata/o,50s,1126,45.359387,11.788713,21.57,20753.0,2.22,28001.0,52.202133
3,Abano Terme,femmine,coniugata/o,60s,1049,45.359387,11.788713,21.57,20753.0,2.22,28001.0,48.632360
4,Abano Terme,femmine,coniugata/o,70s,714,45.359387,11.788713,21.57,20753.0,2.22,28001.0,33.101530
...,...,...,...,...,...,...,...,...,...,...,...,...
50945,Zumpano,maschi,coniugata/o,60s,112,39.310632,16.293506,8.06,11036.0,2.54,78155.0,13.895782
50946,Zumpano,maschi,nubile/celibe,Kid,163,39.310632,16.293506,8.06,11036.0,2.54,78155.0,20.223325
50947,Zumpano,maschi,nubile/celibe,Teen,121,39.310632,16.293506,8.06,11036.0,2.54,78155.0,15.012407
50948,Zumpano,maschi,nubile/celibe,20s,129,39.310632,16.293506,8.06,11036.0,2.54,78155.0,16.004963


## Get only Lombardy

In [41]:
# lombardy = pd.read_excel('New/Elenco-comuni-italiani.xls', sheet_name=0) #reads the first sheet of your excel file
# lombardy = lombardy[lombardy['Denominazione Regione']=='Lombardia'] #Filtering dataframe
# df["ITTER107"] = pd.to_numeric(df["ITTER107"])
# df = pd.merge(df, lombardy,how='inner',left_on=['ITTER107'],right_on=['Codice Comune formato alfanumerico'])
# df.drop(lombardy.columns,axis = 1, inplace = True)
# df

In [42]:
# import seaborn as sns

# heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
# heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [43]:
dfSliced = df[['lng','lat','Sesso','Stato civile','AgeGroup','AvgNumberOfComponents','Redditi 2020']]
catColumnsPos = [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('object').columns)]
catColumnsPos += [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('category').columns)]

In [44]:
dfSliced = pd.get_dummies(dfSliced)
dfSliced.describe()

,lng,lat,AvgNumberOfComponents,Redditi 2020,Sesso_femmine,Sesso_maschi,Stato civile_coniugata/o,Stato civile_divorziata/o,Stato civile_nubile/celibe,Stato civile_vedova/o,...,AgeGroup_30s,AgeGroup_40s,AgeGroup_50s,AgeGroup_60s,AgeGroup_70s,AgeGroup_80s,AgeGroup_90s,AgeGroup_Infant,AgeGroup_Kid,AgeGroup_Teen
count,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,...,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000,50950.000000
mean,11.353424,43.819661,2.392884,18357.597978,0.510893,0.489107,0.486516,0.014485,0.436820,0.062179,...,0.125692,0.153719,0.150697,0.116683,0.097939,0.056546,0.005535,0.024141,0.093405,0.075074
std,2.662191,2.498323,0.204700,3865.324100,0.499886,0.499886,0.499823,0.119479,0.495997,0.241482,...,0.331505,0.360683,0.357757,0.321045,0.297235,0.230975,0.074191,0.153489,0.291002,0.263513
min,7.046420,35.501955,1.630000,7631.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.198760,41.557748,2.280000,15757.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.538235,45.312214,2.380000,18984.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,13.656463,45.651065,2.490000,20900.250000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,18.453989,46.897972,3.120000,43674.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Normalize

In [45]:
# from sklearn import preprocessing
# x = dfSliced[["AvgNumberOfComponents","Redditi 2020"]].values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# dfSliced[["AvgNumberOfComponents","Redditi 2020"]] = pd.DataFrame(
#     x_scaled,columns=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].columns,)

### Standardize

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(dfSliced[["AvgNumberOfComponents","Redditi 2020"]].values)
scaled_features_df = pd.DataFrame(scaled_features, 
                                  index=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].index, 
                                  columns=dfSliced[["AvgNumberOfComponents","Redditi 2020"]].columns)
dfSliced[["AvgNumberOfComponents","Redditi 2020"]] = scaled_features_df

In [47]:
dfSliced

,lng,lat,AvgNumberOfComponents,Redditi 2020,Sesso_femmine,Sesso_maschi,Stato civile_coniugata/o,Stato civile_divorziata/o,Stato civile_nubile/celibe,Stato civile_vedova/o,...,AgeGroup_30s,AgeGroup_40s,AgeGroup_50s,AgeGroup_60s,AgeGroup_70s,AgeGroup_80s,AgeGroup_90s,AgeGroup_Infant,AgeGroup_Kid,AgeGroup_Teen
0,11.788713,45.359387,-0.844582,0.619722,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,11.788713,45.359387,-0.844582,0.619722,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,11.788713,45.359387,-0.844582,0.619722,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,11.788713,45.359387,-0.844582,0.619722,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,11.788713,45.359387,-0.844582,0.619722,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,16.293506,39.310632,0.718697,-1.894193,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
50946,16.293506,39.310632,0.718697,-1.894193,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
50947,16.293506,39.310632,0.718697,-1.894193,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
50948,16.293506,39.310632,0.718697,-1.894193,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# import seaborn as sns

# heatmap = sns.heatmap(dfSliced.corr(), vmin=-1, vmax=1, annot=True)
# heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

## Clustering KMeans + elbow


In [62]:
nClusters = 400 #88 Lombardy #170 Italy

# ss = StandardScaler()
# X = ss.fit_transform(df[['lng','lat']]) #'populPerKm2' per avere cluster di popolazione simile (non voluta)
# dfSliced = pd.concat([pd.DataFrame(X, columns = ['lng','lat']).reset_index(),df[['Sesso','Stato civile','AgeGroup']].reset_index()], axis = 1).drop(["index"], axis = 1)
# print(dfSliced)
kmeans = KMeans( n_clusters = nClusters, init = 'k-means++', random_state = 0)
kMeansCategories = kmeans.fit_predict(dfSliced)
df["cluster"] = kMeansCategories


# ss = StandardScaler()
# X = ss.fit_transform(df[['lng','lat','Sesso','Stato civile','AgeGroup']]) #'populPerKm2' per avere cluster di popolazione simile (non voluta)
# #clustering = DBSCAN(eps=0.1, min_samples=10).fit(X)
# km = KMeans(n_clusters=15,init = 'k-means++', random_state=0, max_iter = 1000)#15
# clustering = km.fit(X)
# np.unique(clustering.labels_)

### Elbow method

In [ ]:
# Choose optimal K using Elbow method
distortions = []
inertias = []

rangeCl = range(10, 100)

print(catColumnsPos)
for cluster in rangeCl:
    kmeans = KMeans( n_clusters = cluster, init = 'k-means++', random_state = 0)
    categories = kmeans.fit_predict(dfSliced)
    distortions.append(sum(np.min(cdist(dfSliced, kmeans.cluster_centers_,
                                        'euclidean'), axis=1)) / dfSliced.shape[0])
    inertias.append(kmeans.inertia_)
    print('Cluster initiation: {}'.format(cluster))

In [ ]:
#Data viz

plt.plot(rangeCl, inertias, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Inertia')
plt.title('The Elbow Method using Inertia')
plt.show()

In [ ]:
plt.plot(rangeCl, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.show()

## Clustering KPrototypes + Elbow

In [ ]:
# from kmodes.kprototypes import KPrototypes

# nClusters = 30 #30

# # ss = StandardScaler()
# # X = ss.fit_transform(df[['lng','lat']]) #'populPerKm2' per avere cluster di popolazione simile (non voluta)
# # dfSliced = pd.concat([pd.DataFrame(X, columns = ['lng','lat']).reset_index(),df[['Sesso','Stato civile','AgeGroup']].reset_index()], axis = 1).drop(["index"], axis = 1)
# # print(dfSliced)
# dfSliced = df[['lng','lat','Sesso','Stato civile','AgeGroup','AvgNumberOfComponents','Redditi 2020']]

# catColumnsPos = [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('object').columns)]
# catColumnsPos += [dfSliced.columns.get_loc(col) for col in list(dfSliced.select_dtypes('category').columns)]
# kprototype = KPrototypes(n_jobs = -1, n_clusters = nClusters, init = 'Huang', random_state = 0)
# categories = kprototype.fit_predict(dfSliced, categorical = catColumnsPos)
# df["cluster"] = categories


# # ss = StandardScaler()
# # X = ss.fit_transform(df[['lng','lat','Sesso','Stato civile','AgeGroup']]) #'populPerKm2' per avere cluster di popolazione simile (non voluta)
# # #clustering = DBSCAN(eps=0.1, min_samples=10).fit(X)
# # km = KMeans(n_clusters=15,init = 'k-means++', random_state=0, max_iter = 1000)#15
# # clustering = km.fit(X)
# # np.unique(clustering.labels_)

## Analyze clusters
##### By changing cluster number and see which people comprises

In [ ]:
# for col in df.columns:
#     if df.columns.get_loc(col) < 4:
#         print(df.loc[df["cluster"] == selectedCluster][col].value_counts())

In [ ]:
for i in range(0,nClusters):#For each cluster
    print("Cluster #" + str(i))
    for col in df.columns: #For each "important" column
        if (col not in ["Value","Territorio","lat","lng","Size km2","populPerKm2","cluster","ITTER107"]):
            print(df.loc[df["cluster"] == i][col].astype(str).value_counts())
    print()#Whitespace

In [ ]:
clusterNames = []
for i in range(0,nClusters):#For each cluster
    clusterName = ""
    for col in df.columns: #For each "important" column
        if (col not in ["Value","Territorio","lat","lng","Size km2","populPerKm2","cluster","ITTER107"]):
            if col == "Redditi 2020":
                clusterName += "Reddito medio " + str(int(df.loc[df["cluster"] == i][col].mean())) + " "
            elif col == "AvgNumberOfComponents":
                clusterName += "Numero medio di componenti " + str(round(df.loc[df["cluster"] == i][col].mean(),2)) + " "
            else:
                clusterName += str(df.loc[df["cluster"] == i][col].value_counts().idxmax()) + " "
    clusterNames.append(clusterName)

In [ ]:
clusterNames
seen = set()
uniq = []
for x in clusterNames:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
print(len(clusterNames))
print(len(seen))

In [ ]:
clusterNames

In [ ]:
# #Choose optimal K using Elbow method
# cost = []

# rangeCl = range(10, 60)

# print(catColumnsPos)
# for cluster in rangeCl:
#     try:
#         kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
#         kprototype.fit_predict(dfSliced, categorical = catColumnsPos)
#         cost.append(kprototype.cost_)
#         print('Cluster initiation: {}'.format(cluster))
#     except:
#         break
# # Converting the results into a dataframe and plotting them
# #Data viz

# %matplotlib inline
# plt.plot( rangeCl, cost, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method using Distortion')
# plt.show()
# #plt.savefig('elbow.png', dpi=fig.dpi)


## Visualization

In [ ]:
# # Extract the data we're interested in
# lat = df['lat'].values
# lon = df['lng'].values
# population = df['Value'].values
# populationsqm = df['populPerKm2'].values
# #area = cities['area_total_km2'].values
selectedCluster = 60
lat = df.loc[df["cluster"] == selectedCluster]['lat'].values
lon = df.loc[df["cluster"] == selectedCluster]['lng'].values
population = df.loc[df["cluster"] == selectedCluster]['Value'].values
populationsqm = df.loc[df["cluster"] == selectedCluster]['populPerKm2'].values
area = df.loc[df["cluster"] == selectedCluster]['Size km2'].values

## Useful functions

In [ ]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def getxyz(event):
    s = ax.format_coord(event.xdata, event.ydata)
    try:
        out = [float(x.split('=')[1].strip()) for x in s.split(',')]
    except:
        out = None
    return out

def onclick(event): #Need to zoom  a bit for it to work (????)
    out = getxyz(event)
    if out != None:
        az = ax.azim
        el = ax.elev
        if (abs(el) <= 2): #If elevation 0
            if (abs(az) <= 2 or abs(abs(az) -180) <= 2): # get z and not x
                zPt = out[2]
                latPt = out[1]
                closest = closest_point((zPt/100,latPt), list(zip(populationsqm/100, lat)))#Get closest point from picked
                tx = '%s' % (df.loc[(df["populPerKm2"] >= (closest[0]*100-0.1)) & 
                                    (df["populPerKm2"] <= (closest[0]*100+0.1))&
                                    (df["cluster"] == selectedCluster)&
                                    (df["lat"] == closest[1])]["Territorio"].values[0])
                ax.set_title(tx)
            elif (abs(az)-90 <= 2):# get z and not y
                zPt = out[2]
                lonPt = out[0]
                tx = out[0]
                closest = closest_point((zPt/100,lonPt), list(zip(populationsqm/100, lon)))#Get closest point from picked
                tx = '%s' % (df.loc[(df["populPerKm2"] >= (closest[0]*100-0.1)) & 
                                    (df["populPerKm2"] <= (closest[0]*100+0.1))&
                                    (df["cluster"] == selectedCluster)&
                                    (df["lng"] == closest[1])]["Territorio"].values[0])
                ax.set_title(tx)
            else:
                ax.set_title("Please adjust azimuth")
        elif (abs(el-90) <= 2):
            lonPt = out[0]#llcrnrlon+(urcrnrlon-llcrnrlon)/2 + event.xdata*((urcrnrlon-llcrnrlon)/2)/0.05481
            latPt = out[1]#llcrnrlat+(urcrnrlat-llcrnrlat)/2 + event.ydata*((urcrnrlat-llcrnrlat)/2)/0.05481
            closest = closest_point((lonPt,latPt), list(zip(lon, lat)))#Get closest point from picked
            tx = '%s' % (df.loc[(df["lng"] == closest[0])&
                                (df["cluster"] == selectedCluster)&
                                (df["lat"] == closest[1])]["Territorio"].values[0])
            ax.set_title(tx)
        else:
            ax.set_title("Please adjust elevation")

In [ ]:
def kmRadius(c): #Define area of attractiveness
    point = Point(c)
    
    local_azimuthal_projection = f"+proj=aeqd +R=6371000 +units=m +lat_0={point.y} +lon_0={point.x}"

    wgs84_to_aeqd = partial(
        pyproj.transform,
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
        pyproj.Proj(local_azimuthal_projection),
    )

    aeqd_to_wgs84 = partial(
        pyproj.transform,
        pyproj.Proj(local_azimuthal_projection),
        pyproj.Proj('+proj=longlat +datum=WGS84 +no_defs'),
    )

    point_transformed = transform(wgs84_to_aeqd, point)

    buffer = point_transformed.buffer(20_000)#Meters

    buffer_wgs84 = transform(aeqd_to_wgs84, buffer)
    dic = mapping(buffer_wgs84)
    return np.asarray(dic['coordinates'])[0]

##Mean based on: municipalities with more target population per sqmt influence more (can be **2 or **3)
##Remove far away points from calculation
def calculateWeightedMean(lat, lon, populationsqm):
    avLat = np.average(a=lat[clf > 0], weights=populationsqm[clf > 0]**3)
    avLon = np.average(a=lon[clf > 0], weights=populationsqm[clf > 0]**3)
    return avLon,avLat


## 3D map Lombardy

In [ ]:
# for i in range (0, 4):
#     # Extract the data we're interested in
#     lat = df.loc[df["cluster"] == i]['lat'].values
#     lon = df.loc[df["cluster"] == i]['lng'].values
#     population = df.loc[df["cluster"] == i]['Value'].values
#     populationsqm = df.loc[df["cluster"] == i]['populPerKm2'].values
#     area = df.loc[df["cluster"] == i]['Size km2'].values
fig = plt.figure(0)
ax = fig.add_subplot(projection='3d')
# print(clusterNames[selectedCluster])

llcrnrlat = 35.5#44
urcrnrlat = 47
llcrnrlon = 7#7
urcrnrlon = 18.5#12
# Create a basemap instance that draws the Earth layer
bm = Basemap(llcrnrlat=llcrnrlat, llcrnrlon=llcrnrlon, 
            urcrnrlat=urcrnrlat, urcrnrlon=urcrnrlon,
            projection='cyl', resolution='l', fix_aspect=False, ax=ax)
# Add Basemap to the figure
ax.add_collection3d(bm.drawcoastlines(linewidth=0.25))
ax.add_collection3d(bm.drawcountries(linewidth=0.35))
ax.view_init(azim=-90, elev=90)
ax.set_xlabel('Longitude (°E)', labelpad=20)
ax.set_ylabel('Latitude (°N)', labelpad=20)
ax.set_zlabel('Population per sqkm', labelpad=20)
ax.set_zlim(0., 400)

colormap = np.arange(0,15)

from sklearn.ensemble import IsolationForest

clf = IsolationForest( random_state=0).fit_predict(np.array((lon,lat)).T)#Schema a blochi pptx

lonC, latC = calculateWeightedMean(lat, lon, populationsqm)
p = ax.scatter(lon, lat, (populationsqm), picker = True, s = area,c=colormap[clf])#Points # c=colormap[categories]
centers = ax.scatter(xs=lonC, ys=latC, edgecolor='black', c='white', marker = '*')#Centers

circle = kmRadius((lonC,latC))
r = ax.scatter(xs=circle[:,0], ys=circle[:,1], c='black', marker = '1')#Radius

closest = closest_point((lonC,latC), list(zip(lon, lat)))#Get closest point from picked

tx = 'Center is located in: %s' % (df.loc[(df["lng"] == closest[0])&
                                          (df["cluster"] == selectedCluster)&
                                          (df["lat"] == closest[1])]["Territorio"].values[0])
ax.set_title(tx)
print(tx)
cid = fig.canvas.mpl_connect('button_press_event', onclick)#Get mouse click

fig.show
print(clusterNames[selectedCluster])

In [ ]:
df.loc[(df["cluster"] == selectedCluster) & (df["Territorio"] == "Cusano Milanino")]

In [ ]:
# df.loc[(df["cluster"] == selectedCluster) & (df["Territorio"] == "Lecco")]

## Elbow

In [ ]:
# from scipy.spatial.distance import cdist

# distortions = []
# inertias = []
# mapping1 = {}
# mapping2 = {}
# K = range(2, 20)
 
# for k in K:
#     # Building and fitting the model
#     kmeanModel = KMeans(n_clusters=k).fit(X)
#     kmeanModel.fit(X)
 
#     distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
#                                         'euclidean'), axis=1)) / X.shape[0])
#     inertias.append(kmeanModel.inertia_)
 
#     mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
#                                    'euclidean'), axis=1)) / X.shape[0]
#     mapping2[k] = kmeanModel.inertia_

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('Values of K')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method using Distortion')
# plt.show()

## Add transport (trains)

### Regionals

In [ ]:
stops = pd.read_csv("new/Orario_Ferroviario_Regionale_-_Stops.csv")
stops= stops.drop(["stop_code","stop_id", "stop_desc", "stop_url", "location_type", "parent_station","location"],axis = 1)
stops

In [ ]:
def cutAwayMoreThanMinutes(stops):
#     avgTrainSpeed = 45 #km/h 
    #Remove if more than 1.5h
    #Remove from Colico and from Iseo
    point = stops.loc[stops["stop_name"] == "COLICO"]
    stops = stops.drop(stops[stops["stop_lat"] > point["stop_lat"].values[0]].index)#Prata Camportaccio
    stops = stops.drop(stops[(stops["stop_lat"] > point["stop_lat"].values[0]-0.1) & 
                       (stops["stop_lon"] > point["stop_lon"].values[0])].index)#Tirano
    point = stops.loc[stops["stop_name"] == "ISEO"]
    stops = stops.drop(stops[(stops["stop_lat"] > point["stop_lat"].values[0]) & 
                       (stops["stop_lon"] > point["stop_lon"].values[0])].index)#Cedegolo
    return stops


In [ ]:
stops = cutAwayMoreThanMinutes(stops)
p = ax.scatter(xs=stops["stop_lon"], ys=stops["stop_lat"],marker = '.', c = "Black")#Centers
display(fig)  # Shows plot again

# r.remove()
# centers.remove()

## High speed

In [ ]:
hs = pd.read_csv("new/Orario_Ferroviario_Frecce.csv")
hs

In [ ]:
milano = stops.loc[stops["stop_name"] =="MILANO CENTRALE"]
for index, row in hs.iterrows():
    ax.plot([row["stop_lon"],milano["stop_lon"].values[0]], [row["stop_lat"],milano["stop_lat"].values[0]])

display(fig)  # Shows plot again


## Compare clustering results

#TODO attach revenues to clustering
clustering: 
homogeneity
adjusted_mutual_info_score
adjusted_rand_score
calinski_harabasz_score # Higher better
davies_bouldin_score  # 0 best
silhouette_score   # 1 best, -1 worst

In [95]:
HomogenScore = []
MutualInfoScore = []
AdjustedRandScore = []

CHScore = []
DBScore = []
SilhouetteScore = []

In [96]:
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import calinski_harabasz_score
from sklearn.metrics.cluster import davies_bouldin_score
from sklearn.metrics.cluster import silhouette_score

# homogeneity_score(kMeansCategories,Y)
# adjusted_mutual_info_score(kMeansCategories, [0, 0, 1, 1])
# adjusted_rand_score(kMeansCategories, [0, 0, 1, 1])
calinski_harabasz_score (dfSliced,kMeansCategories)
davies_bouldin_score(dfSliced,kMeansCategories)
silhouette_score(dfSliced,kMeansCategories,sample_size = int(len(dfSliced)/5),random_state = 0) 

CHScore.append(["KMeans with Elbow",calinski_harabasz_score (dfSliced,kMeansCategories)])
DBScore.append(["KMeans with Elbow",davies_bouldin_score(dfSliced,kMeansCategories)])
SilhouetteScore.append(["KMeans with Elbow",silhouette_score(dfSliced,kMeansCategories,
                                                             sample_size = int(len(dfSliced)/5),random_state = 0)])

In [89]:
pd.DataFrame(CHScore, columns = ['Algorithm', 'CH Value'])

,Algorithm,CH Value
0,KMeans with Elbow,2483.419453


In [97]:
pd.DataFrame(DBScore, columns = ['Algorithm', 'DB Value'])

,Algorithm,DB Value
0,KMeans with Elbow,1.45625


In [98]:
pd.DataFrame(SilhouetteScore, columns = ['Algorithm', 'Silhouette value'])

,Algorithm,Silhouette value
0,KMeans with Elbow,0.231112
